## Данные

Тут - https://drive.google.com/drive/folders/1Kj9Nk2e1ncV2WYSbfXkL0khBWYLwRoPx?usp=sharing

...или тут - /cephfs/home/mikhaylova/multimodal_emo_reco/MELD_csv/

---
Столбцы:
* таргет_эмоция_число
* таргет_эмоция_категория
* текст_предсказ_эмоция_вероятности
* аудио_предсказ_эмоция_вероятности
* видео_предсказ_эмоция_вероятности
---

Обучающая: **9989**

Тестовая: **2610**

Валидационная: **1109**

! *Если видео отсутствовало - предсказание равновероятно для каждого из классов*

## Wandb

In [ ]:
! pip install wandb

In [4]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
sweep_config = {'method': 'grid'}

metric = {
    'name': 'val F1',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

In [6]:
# variable parameters
parameters_dict = {
        'lr': {
            'values': [1e-03, 1e-04]
        },
        'seed': {
            'values': [13, 42, 100, 47]
        }
}

sweep_config['parameters'] = parameters_dict

In [7]:
parameters_dict.update({
    'epochs': {
        'value': 10
    },
    'batch_size': {
        'value': 4
    },
})

In [8]:
sweep_id = wandb.sweep(sweep_config, project="simple-multimodal-model")

Create sweep with ID: j8a5507j
Sweep URL: https://wandb.ai/annette-mikhaylova/simple-multimodal-model/sweeps/j8a5507j


## Model architecture

In [9]:
import torch.nn as nn
import torch


class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()

    # number of features (len of X cols)
    input_dim = 21
    # number of hidden layers
    hidden_layers = 64
    # number of classes (unique of y)
    output_dim = 7
    
    self.linear1 = nn.Linear(input_dim, hidden_layers)
    self.linear2 = nn.Linear(hidden_layers, output_dim)
    self.drop_out = nn.Dropout(0.2)

  def forward(self, x):
    x = torch.sigmoid(self.linear1(x))
    x = self.drop_out(x)
    x = self.linear2(x)
    return x

## Training with Wandb

In [11]:
import numpy as np
import random
from sklearn.metrics import f1_score

from loading_model_logits import prepare_data
from utils import Data
from training_model import train, get_preds

def train_net(config=None):
    with wandb.init(config=config) as run:

        config = wandb.config
        torch.manual_seed(config.seed)
        random.seed(config.seed)
        np.random.seed(config.seed)

        epochs = config.epochs
        name_str = f"2lay_{config.seed}_lr-{config.lr}"
        run.name = name_str

        X_train, y_train, X_dev, y_dev, X_test, y_test = prepare_data(
            '/content/drive/MyDrive/MeldCSV/'
            )
        train_data, dev_data, test_data =   Data(X_train, y_train),\
                                            Data(X_dev, y_dev), \
                                            Data(X_test, y_test)

        trainloader = DataLoader(train_data, batch_size=config.batch_size, 
                                 shuffle=True, num_workers=2)
        devloader = DataLoader(dev_data, batch_size=config.batch_size, 
                               shuffle=True, num_workers=2)
        testloader = DataLoader(test_data, batch_size=config.batch_size, 
                               shuffle=True, num_workers=2)

        net = Network()
        optimizer = torch.optim.Adam(net.parameters(), lr=config.lr)
        loss_fn = torch.nn.CrossEntropyLoss()

        train(epochs, net, trainloader, devloader, optimizer, loss_fn, 3)

        test_pred = get_preds(X_test, y_test, net)
        f1 = f1_score(y_test.cpu().numpy(), test_pred.cpu().numpy(), 
                              average='weighted')
        wandb.log({"test f1": f1})

In [12]:
wandb.agent(sweep_id, train_net)

wandb: Agent Starting Run: lrjwjo9g with config:
wandb: 	batch_size: 4
wandb: 	epochs: 10
wandb: 	lr: 0.001
wandb: 	seed: 13
wandb: Currently logged in as: annette-mikhaylova. Use `wandb login --relogin` to force relogin


batch train loss,▇▅▅▅▁▂▇▃▅▂▁▄▁▁▂▂▄▂█▃▇▅▃▂▄▁▂▃▁▁▂▃▁▂▄▄▅▂▁▁
epoch,▁▂▄▅▇█
test f1,▁
train F1,▁▇████
train loss,█▃▂▁▁▁
val F1,▁▅█▄▂▅
val loss,█▃▂▂▁▁
batch train loss,1.2123
epoch,5
test f1,0.62807
train F1,0.84488


wandb: Agent Starting Run: uuygl3io with config:
wandb: 	batch_size: 4
wandb: 	epochs: 10
wandb: 	lr: 0.001
wandb: 	seed: 42


batch train loss,█▃▄▂▄▆▄▁▂▁▄▅▁▁▁▁▄▃▃▁▃▂▂▄▁▂▁▄▁▃▁▂▃▁▁▄▂▅▁▄
epoch,▁▂▃▅▆▇█
test f1,▁
train F1,▁▇▇████
train loss,█▃▂▂▁▁▁
val F1,▁▂█▅▇▃▇
val loss,█▄▂▁▂▁▂
batch train loss,1.16566
epoch,6
test f1,0.62865
train F1,0.84838


wandb: Agent Starting Run: i1nlc5k5 with config:
wandb: 	batch_size: 4
wandb: 	epochs: 10
wandb: 	lr: 0.001
wandb: 	seed: 100


...

wandb: Agent Starting Run: tq70ujzh with config:
wandb: 	batch_size: 4
wandb: 	epochs: 10
wandb: 	lr: 0.0001
wandb: 	seed: 100


batch train loss,█▆▇▇▄▆▅▃▃▂▁▆▂▃▃▃▂▂▂▃▅▄▃▄▂▃▃▃▃▃▂▂▁▂▂▂▁▃▃▂
epoch,▁▂▃▃▄▅▆▆▇█
test f1,▁
train F1,▁▂▄▆▇█████
train loss,█▅▄▃▂▂▂▁▁▁
val F1,▁▂▅▆▇█████
val loss,█▅▄▃▂▂▂▁▁▁
batch train loss,1.26746
epoch,9
test f1,0.61736
train F1,0.80736


wandb: Agent Starting Run: ydiqw3dc with config:
wandb: 	batch_size: 4
wandb: 	epochs: 10
wandb: 	lr: 0.0001
wandb: 	seed: 47


batch train loss,█▇▅▆▄▄▃▂▆▇▄▂▄▄▃▃▂▃▂▃▅▃▂▃▂▃▆▄▃▄▄▂▅▄▂▂▃▂▃▁
epoch,▁▂▃▄▅▅▆▇█
test f1,▁
train F1,▁▃▅▇▇████
train loss,█▅▃▃▂▂▁▁▁
val F1,▁▃▆▆▇▇███
val loss,█▅▄▃▂▂▁▁▁
batch train loss,1.56528
epoch,8
test f1,0.61496
train F1,0.80116


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
